In [1]:
import pandas as pd
import numpy as np
import janitor as jn

In [2]:
df = {
    "A": [0, 0, 0, 0],
    "E": pd.to_datetime(
        [
            "1970-01-01 00:00:00.000000001",
            "1970-01-01 00:00:00.000000000",
            "1969-12-31 23:59:59.999999999",
            "1970-01-01 00:00:00.000000000",
        ]
    ),
}


right = {
    "Dates": pd.to_datetime([
        "1970-01-01 00:00:00.000000000",
        "1970-01-01 00:00:00.000000001",
        "1970-01-01 00:00:00.000000000",
    ]),
    "Dates_Right": pd.to_datetime(
        [
            "1970-01-01 00:00:00.000000001",
            "1969-12-31 23:59:59.999999999",
            "1969-12-31 23:59:59.999999999",
        ]
    ),
}


right = pd.DataFrame(right)

df = pd.DataFrame(df)

In [3]:
df

,A,E
0,0,1970-01-01 00:00:00.000000001
1,0,1970-01-01 00:00:00.000000000
2,0,1969-12-31 23:59:59.999999999
3,0,1970-01-01 00:00:00.000000000


In [4]:
right

,Dates,Dates_Right
0,1970-01-01 00:00:00.000000000,1970-01-01 00:00:00.000000001
1,1970-01-01 00:00:00.000000001,1969-12-31 23:59:59.999999999
2,1970-01-01 00:00:00.000000000,1969-12-31 23:59:59.999999999


In [5]:
A =(
    df[["E"]].conditional_join(
        right[["Dates", "Dates_Right"]],
        ("E", "Dates", ">="),
        ("E", "Dates_Right", "<="),
        how="inner",
        use_numba=True,
    )
)

A

,E,Dates,Dates_Right
0,1970-01-01 00:00:00.000000001,1970-01-01,1970-01-01 00:00:00.000000001
1,1970-01-01 00:00:00.000000000,1970-01-01,1970-01-01 00:00:00.000000001
2,1970-01-01 00:00:00.000000000,1970-01-01,1970-01-01 00:00:00.000000001


In [6]:
B = (
    df[["E"]].conditional_join(
        right[["Dates", "Dates_Right"]],
        ("E", "Dates", ">="),
        ("E", "Dates_Right", "<="),
        how="inner",
        use_numba=False,
    )
)

B

,E,Dates,Dates_Right
0,1970-01-01 00:00:00.000000001,1970-01-01,1970-01-01 00:00:00.000000001
1,1970-01-01 00:00:00.000000000,1970-01-01,1970-01-01 00:00:00.000000001
2,1970-01-01 00:00:00.000000000,1970-01-01,1970-01-01 00:00:00.000000001


In [7]:
A.sort_values(["E", "Dates", "Dates_Right"], ignore_index=True)

,E,Dates,Dates_Right
0,1970-01-01 00:00:00.000000000,1970-01-01,1970-01-01 00:00:00.000000001
1,1970-01-01 00:00:00.000000000,1970-01-01,1970-01-01 00:00:00.000000001
2,1970-01-01 00:00:00.000000001,1970-01-01,1970-01-01 00:00:00.000000001


In [8]:
C.sort_values(["E", "Dates", "Dates_Right"], ignore_index=True)

NameError: name 'C' is not defined

In [ ]:
B.sort_values(["E", "Dates", "Dates_Right"], ignore_index=True)

,E,Dates,Dates_Right
0,1969-12-31 23:59:59.999999999,1969-12-31 23:59:59.999999999,1970-01-01 00:00:00.000000000
1,1969-12-31 23:59:59.999999999,1969-12-31 23:59:59.999999999,1970-01-01 00:00:00.000000001
2,1970-01-01 00:00:00.000000000,1969-12-31 23:59:59.999999999,1970-01-01 00:00:00.000000000
3,1970-01-01 00:00:00.000000000,1969-12-31 23:59:59.999999999,1970-01-01 00:00:00.000000001
4,1970-01-01 00:00:00.000000001,1969-12-31 23:59:59.999999999,1970-01-01 00:00:00.000000001
5,1970-01-01 00:00:00.000000001,1969-12-31 23:59:59.999999999,1970-01-01 00:00:00.000000001


In [ ]:
C = (
    df[["E"]]
    .merge(right[["Dates", "Dates_Right"]], how="cross")
    .loc[lambda df: df.E.between(df.Dates, df.Dates_Right, inclusive="both")]
)

C

,E,Dates,Dates_Right
1,1970-01-01 00:00:00.000000001,1969-12-31 23:59:59.999999999,1970-01-01 00:00:00.000000001
4,1970-01-01 00:00:00.000000000,1969-12-31 23:59:59.999999999,1970-01-01 00:00:00.000000001
5,1970-01-01 00:00:00.000000000,1969-12-31 23:59:59.999999999,1970-01-01 00:00:00.000000000
7,1969-12-31 23:59:59.999999999,1969-12-31 23:59:59.999999999,1970-01-01 00:00:00.000000001
8,1969-12-31 23:59:59.999999999,1969-12-31 23:59:59.999999999,1970-01-01 00:00:00.000000000
10,1970-01-01 00:00:00.000000000,1969-12-31 23:59:59.999999999,1970-01-01 00:00:00.000000001
11,1970-01-01 00:00:00.000000000,1969-12-31 23:59:59.999999999,1970-01-01 00:00:00.000000000


In [ ]:
C.sort_values(["E", "Dates", "Dates_Right"], ignore_index=True)

,E,Dates,Dates_Right
0,1969-12-31 23:59:59.999999999,1969-12-31 23:59:59.999999999,1970-01-01 00:00:00.000000000
1,1969-12-31 23:59:59.999999999,1969-12-31 23:59:59.999999999,1970-01-01 00:00:00.000000001
2,1970-01-01 00:00:00.000000000,1969-12-31 23:59:59.999999999,1970-01-01 00:00:00.000000000
3,1970-01-01 00:00:00.000000000,1969-12-31 23:59:59.999999999,1970-01-01 00:00:00.000000001
4,1970-01-01 00:00:00.000000001,1969-12-31 23:59:59.999999999,1970-01-01 00:00:00.000000001
5,1970-01-01 00:00:00.000000001,1969-12-31 23:59:59.999999999,1970-01-01 00:00:00.000000001


In [ ]:
df.dtypes, right.dtypes

(A             int64
 E    datetime64[ns]
 dtype: object,
 Integers             int64
 Dates       datetime64[ns]
 dtype: object)

In [ ]:
(
    df[["A", "E"]].conditional_join(
        right[["Integers", "Dates"]],
        ("E", "Dates", "<"),
        ("A", "Integers", ">="),
        how="inner",
        use_numba=True,
        sort_by_appearance=False,
    )
)

KeyError: "['Integers'] not in index"

In [ ]:
columns = ["A", "E", "Integers", "Dates"]
(
    df.merge(
        right,
        how="cross",
    )
    .loc[lambda df: df.A.ge(df.Integers) & df.E.lt(df.Dates), columns]
    .sort_values(columns, ignore_index=True)
)

,A,E,Integers,Dates
0,-256,1969-12-31 23:59:59.999998710,-50666597047599360,1969-12-31 23:59:59.999999996
1,-256,1969-12-31 23:59:59.999998710,-16843521,1969-12-31 23:59:59.999999996


In [ ]:
(
    df[["A", "E"]].conditional_join(
        right[["Integers", "Dates"]],
        ("E", "Dates", "<"),
        ("A", "Integers", ">="),
        how="inner",
        use_numba=False,
        sort_by_appearance=False,
    )
)

,A,E,Integers,Dates
0,-256,1969-12-31 23:59:59.999998710,-50666597047599360,1969-12-31 23:59:59.999999996
1,-256,1969-12-31 23:59:59.999998710,-16843521,1969-12-31 23:59:59.999999996


In [ ]:
print 'j'

SyntaxError: Missing parentheses in call to 'print'. Did you mean print('j')? (427054430.py, line 1)

In [ ]:
pd.__version__

'2.2.1'

In [ ]:
df1 = pd.DataFrame({"id": [1, 1, 1, 2, 2, 3], "value_1": [2, 5, 7, 1, 3, 4]})
df2 = pd.DataFrame(
    {
        "id": [1, 1, 1, 1, 2, 2, 2, 3],
        "value_2A": [0, 3, 7, 12, 0, 2, 3, 1],
        "value_2B": [1, 5, 9, 15, 1, 4, 6, 3],
    }
)

In [ ]:
df1 = pd.concat([df1] * 20000)
df2 = pd.concat([df2] * 300)

df1.conditional_join(
    df2,
    ("value_1", "value_2A", ">"),
    ("value_1", "value_2B", "<"),
    use_numba=True,
)

(array([2, 3, 2, ..., 0, 0, 0]),
 array([3, 3, 3, ..., 0, 0, 0]),
 array([2398, 2390, 2382, ...,   23,   15,    7]),
 array([600, 300, 600, ..., 300, 300, 300]),
 array([1200, 1200, 1200, ...,  300,  300,  300]),
 array([  0,   0,   0, ..., 900, 900, 900]),
 30000000)

In [ ]:
df1.conditional_join(
    df2,
    ("value_1", "value_2A", ">"),
    ("value_1", "value_2B", "<"),
    use_numba=False,
)

In [ ]:
%timeit df1.conditional_join(df2, ('value_1','value_2A','>'), ('value_1','value_2B','<'),use_numba=True)

29.9 ms ± 163 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
# 32.6ms   30_000_000

In [ ]:
df1

,id,value_1
0,1,2
1,1,5
2,1,7
3,2,1
4,2,3
...,...,...
1,1,5
2,1,7
3,2,1
4,2,3


In [34]:
events = pd.read_csv(
    "/home/sam/Downloads/results.csv", parse_dates=["start", "end"]
).iloc[:, 1:]
events.head()

,id,name,audience,start,sponsor,end
0,1,Event 1,1178,2022-11-19 10:00:00,Sponsor 2,2022-11-19 10:15:00
1,2,Event 2,1446,2015-09-27 15:00:00,Sponsor 11,2015-09-27 15:11:00
2,3,Event 3,2261,2019-11-12 18:00:00,Sponsor 10,2019-11-12 18:53:00
3,4,Event 4,1471,2019-12-24 22:00:00,Sponsor 6,2019-12-24 22:11:00
4,5,Event 5,2605,2028-06-20 12:00:00,Sponsor 8,2028-06-20 12:31:00


In [ ]:
# events = pd.read_csv('/Users/samuel.oranyeli/Downloads/results.csv', parse_dates=['start','end']).iloc[:, 1:]

In [10]:
A = events.conditional_join(
    events,
    ("start", "end", "<="),
    ("end", "start", ">="),
    # ('id', 'id', '!='),
    # ('audience','audience','>'),
    use_numba=True,
    df_columns=["id", "start", "end"],
    right_columns=["id", "start", "end"],
)

A

left                                          right  \
          id               start                 end     id   
0      25297 1992-01-01 10:00:00 1992-01-01 10:50:00  25297   
1      10727 1992-01-02 10:00:00 1992-01-02 10:06:00  10727   
2      24104 1992-01-02 11:00:00 1992-01-02 11:11:00  24104   
3      23159 1992-01-03 03:00:00 1992-01-03 05:00:00  23159   
4      23159 1992-01-03 03:00:00 1992-01-03 05:00:00  16937   
...      ...                 ...                 ...    ...   
33696   4591 2031-12-18 04:00:00 2031-12-18 04:38:00   4591   
33697   6163 2031-12-18 22:00:00 2031-12-18 22:46:00   6163   
33698   3545 2031-12-20 10:00:00 2031-12-20 10:49:00   3545   
33699  15885 2031-12-20 21:00:00 2031-12-20 21:09:00  15885   
33700  13368 2031-12-21 01:00:00 2031-12-21 01:13:00  13368   

                                               
                    start                 end  
0     1992-01-01 10:00:00 1992-01-01 10:50:00  
1     1992-01-02 10:00:00 1992-01-02 10:06:00  
2     1992-01-02 11:00:00 1992-01-02 11:11:00  
3     1992-01-03 03:00:00 1992-01-03 05:00:00  
4     1992-01-03 05:00:00 1992-01-03 05:39:00  
...                   ...                 ...  
33696 2031-12-18 04:00:00 2031-12-18 04:38:00  
33697 2031-12-18 22:00:00 2031-12-18 22:46:00  
33698 2031-12-20 10:00:00 2031-12-20 10:49:00  
33699 2031-12-20 21:00:00 2031-12-20 21:09:00  
33700 2031-12-21 01:00:00 2031-12-21 01:13:00  

[33701 rows x 6 columns]

In [ ]:
B = events.conditional_join(
    events,
    ("start", "end", "<="),
    ("end", "start", ">="),
    # ('id', 'id', '!='),
    # ('audience','audience','>'),
    use_numba=False,
    df_columns=["id", "start", "end"],
    right_columns=["id", "start", "end"],
)

In [ ]:
A.sort_values(A.columns.tolist(), ignore_index=True).equals(
    B.sort_values(A.columns.tolist(), ignore_index=True)
)

True

In [35]:
%%timeit
(
    events.conditional_join(
        events,
        ("start", "end", "<="),
        ("end", "start", ">="),
        # ('id', 'id', '!='),
        # ('audience','audience','>'),
        use_numba=True,
        df_columns=["id", "start", "end"],
        right_columns=["id", "start", "end"],
    )
)

28.5 ms ± 1.99 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [36]:
%%timeit
(
    events.conditional_join(
        events,
        ("start", "end", "<="),
        ("end", "start", ">="),
        # ('id', 'id', '!='),
        # ('audience','audience','>'),
        use_numba=False,
        df_columns=["id", "start", "end"],
        right_columns=["id", "start", "end"],
    )
)

34.3 ms ± 1.83 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [28]:
C = dict(
    key=[f"c{num}" for num in range(1, 8)],
    vol=[35, 15, 5, 35, 18, 90, 17],
    profit=[45, 35, 55, 12, 15, 55, 11],
    unitsSold=[15, 10, 30, 10, 15, 80, 2],
    keyy=range(1, 8),
)
C = pd.DataFrame(C)


D = dict(
    key=[f"d{num}" for num in range(1, 9)],
    vol=[20, 50, 15, 16, 40, 20, 40, 2],
    profit=[30, 10, 12, 52, 35, 20, 30, 57],
    unitsSold=[20, 35, 10, 12, 40, 30, 5, 15],
    keyy=range(8, 0, -1),
)
D = pd.DataFrame(D)

In [12]:
out = C.conditional_join(
    D,
    ("profit", "profit", ">="),
    # ('keyy','keyy','=='),
    #  ('unitsSold','unitsSold','>='),
    ("vol", "vol", "<="),
    # ('vol','profit','>'),
    use_numba=True,
)

out

left                           right                          
    key vol profit unitsSold keyy   key vol profit unitsSold keyy
0    c3   5     55        30    3    d5  40     35        40    4
1    c3   5     55        30    3    d7  40     30         5    2
2    c3   5     55        30    3    d2  50     10        35    7
3    c3   5     55        30    3    d1  20     30        20    8
4    c3   5     55        30    3    d6  20     20        30    3
5    c3   5     55        30    3    d4  16     52        12    5
6    c3   5     55        30    3    d3  15     12        10    6
7    c1  35     45        15    1    d5  40     35        40    4
8    c1  35     45        15    1    d7  40     30         5    2
9    c1  35     45        15    1    d2  50     10        35    7
10   c2  15     35        10    2    d5  40     35        40    4
11   c2  15     35        10    2    d7  40     30         5    2
12   c2  15     35        10    2    d2  50     10        35    7
13   c2  15     35        10    2    d1  20     30        20    8
14   c2  15     35        10    2    d6  20     20        30    3
15   c2  15     35        10    2    d3  15     12        10    6
16   c4  35     12        10    4    d2  50     10        35    7
17   c5  18     15        15    5    d2  50     10        35    7
18   c7  17     11         2    7    d2  50     10        35    7

In [13]:
(
    C.conditional_join(
        D,
        ("profit", "profit", ">="),
        # ('keyy','keyy','=='),
        #  ('unitsSold','unitsSold','>='),
        ("vol", "vol", "<="),
        # ('vol','profit','>'),
        use_numba=False,
    )
)

left                           right                          
    key vol profit unitsSold keyy   key vol profit unitsSold keyy
0    c1  35     45        15    1    d2  50     10        35    7
1    c1  35     45        15    1    d7  40     30         5    2
2    c1  35     45        15    1    d5  40     35        40    4
3    c2  15     35        10    2    d2  50     10        35    7
4    c2  15     35        10    2    d3  15     12        10    6
5    c2  15     35        10    2    d6  20     20        30    3
6    c2  15     35        10    2    d1  20     30        20    8
7    c2  15     35        10    2    d7  40     30         5    2
8    c2  15     35        10    2    d5  40     35        40    4
9    c3   5     55        30    3    d2  50     10        35    7
10   c3   5     55        30    3    d3  15     12        10    6
11   c3   5     55        30    3    d6  20     20        30    3
12   c3   5     55        30    3    d1  20     30        20    8
13   c3   5     55        30    3    d7  40     30         5    2
14   c3   5     55        30    3    d5  40     35        40    4
15   c3   5     55        30    3    d4  16     52        12    5
16   c4  35     12        10    4    d2  50     10        35    7
17   c5  18     15        15    5    d2  50     10        35    7
18   c7  17     11         2    7    d2  50     10        35    7

In [29]:
%%timeit
(
    C.conditional_join(
        D,
        ("profit", "profit", ">="),
        # ('keyy','keyy','=='),
        #  ('unitsSold','unitsSold','>='),
        ("vol", "vol", "<="),
        # ('vol','profit','>'),
        use_numba=True,
    )
)

3.38 ms ± 594 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [30]:
%%timeit
(
    C.conditional_join(
        D,
        ("profit", "profit", ">="),
        # ('keyy','keyy','=='),
        #  ('unitsSold','unitsSold','>='),
        ("vol", "vol", "<="),
        # ('vol','profit','>'),
        use_numba=False,
    )
)

3.75 ms ± 320 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
print 'i'

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(...)? (414025362.py, line 1)

In [31]:
C = pd.concat([C] * 400)
D = pd.concat([D] * 400)

In [ ]:
(
    C.conditional_join(
        D,
        ("profit", "profit", ">="),
        # ('keyy','keyy','=='),
        # ('unitsSold','unitsSold','>='),
        ("vol", "vol", "<="),
        # ('vol','profit','>'),
        use_numba=True,
    )
)

(False, False)

In [32]:
%%timeit
(
    C.conditional_join(
        D,
        ("profit", "profit", ">="),
        # ('keyy','keyy','=='),
        # ('unitsSold','unitsSold','>='),
        ("vol", "vol", "<="),
        # ('vol','profit','>'),
        use_numba=True,
    )
)

142 ms ± 4.38 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [33]:
%%timeit
(
    C.conditional_join(
        D,
        ("profit", "profit", ">="),
        # ('keyy','keyy','=='),
        # ('unitsSold','unitsSold','>='),
        ("vol", "vol", "<="),
        # ('vol','profit','>'),
        use_numba=False,
    )
)

175 ms ± 14.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
(
    C.conditional_join(
        D,
        ("profit", "profit", ">="),
        # ('keyy','keyy','=='),
        # ('unitsSold','unitsSold','>='),
        ("vol", "vol", "<="),
        # ('vol','profit','>'),
        use_numba=True,
    )
)[0].size

30400

In [24]:
np.asarray(1), np.asanyarray(1)

(array(1), array(1))

In [ ]:
(
    C.conditional_join(
        D,
        ("profit", "profit", ">="),
        # ('keyy','keyy','=='),
        # ('unitsSold','unitsSold','>='),
        ("vol", "vol", "<="),
        # ('vol','profit','>'),
        use_numba=False,
    )
)[0].size

30400

In [25]:
from string import ascii_lowercase

np.random.seed(1)
# n = 10000000; k = 20000
n = 20_0_000
k = 20_0
mapp = {k: v for v, k in enumerate(ascii_lowercase)}
idx1 = np.random.randint(0, high=2_00, size=n)
idx2 = np.random.randint(0, high=3_00, size=n)

d1 = dict(
    x=np.random.choice(list(ascii_lowercase[:5]), size=n),
    start=np.minimum(idx1, idx2),
    end=np.maximum(idx1, idx2),
)


d2 = dict(
    x=np.random.choice(list(ascii_lowercase[:15]), size=k),
    pos1=np.random.randint(low=60, high=151, size=k),
)

d1 = pd.DataFrame(d1)
d2 = pd.DataFrame(d2)
d1 = d1.assign(xx=lambda df: df.x.map(mapp))
d2 = d2.assign(xx=lambda df: df.x.map(mapp))

In [26]:
%%timeit
bla = d2.conditional_join(
    d1,
    ("pos1", "start", ">"),
    ("pos1", "end", "<"),
    # ('xx', 'xx', '=='),
    use_numba=True,
)

731 ms ± 49.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
%%timeit
bla = d2.conditional_join(
    d1,
    ("pos1", "start", ">"),
    ("pos1", "end", "<"),
    # ('xx', 'xx', '=='),
    use_numba=False,
)

1.27 s ± 30 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%%timeit
bla = d2.conditional_join(
    d1,
    ("pos1", "start", ">"),
    ("pos1", "end", "<"),
    # ('xx', 'xx', '=='),
    use_numba=False,
)

372 ms ± 2.41 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
print 'jarr'

In [ ]:
employees = pd.read_csv("/Users/samuel.oranyeli/employees.csv")

In [ ]:
%%timeit
(
    employees.conditional_join(
        employees, ("salary", "salary", "<"), ("tax", "tax", ">"), use_numba=True
    )
)

In [ ]:
%%timeit
(
    employees.conditional_join(
        employees, ("salary", "salary", "<"), ("tax", "tax", ">"), use_numba=False
    )
)

4.02 s ± 8.26 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
